# Hier baue ich einen Rezeptgenerator basierend auf einem besseren Tokenizer

Der Tokenizer soll
* Komplett umkehrbar sein: Tokenizer-Decoder
* Damit kann ich die Korrektheit überprüfen
* Und die generierten Rezepte haben die volle Ausdrucksfähigkeit
* Groß- und Kleinschreibung wird über [LOWER][UPPER] Token kodiert
* Stemming wird durchgeführt
* Punctuation wird als special tokens separiert
* Whitespace wird über [WHITESPACE] und [ANTISPACE] kodiert

Die Rezepte
* Enden mit [STOP]
* Die Instructions starten mit [INSTRUCTIONS]
* Komische Whitespaces werden gecleant

In [7]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import time

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
#from tensorflow.keras.layers import TextVectorization
import os
import re
import string
import random
import json

In [8]:
def punctuation(string):
    """
    Setzt strategisch token und whitespace ein 
    damit nachher durch .split() auch die Satzzeichen zu Token werden.
    """
    string = string.replace('\r\n\r\n','∆') # multi bytes ersetzen
    string = string.replace('\r\n','Ω') # multi bytes ersetzen
    string = string.replace('\n','ª') # multi bytes ersetzen
    string = string.replace('\r','µ') # multi bytes ersetzen
    
    substrings = ['(', ')', '/', '&', '-', ',', '.', '!', '?', '"', '°', '∆',';',':','Ω','ª','µ']
    
    new_string = ''
    for i,b in enumerate(string):
        if b in substrings:
            if i>0 and string[i-1]==' ':
                new_string += b
            else: 
                new_string += ' [ANTISPACE] '
                new_string += b
        elif b == ' ':
            if i==0 or string[i-1]==' ' or i==len(string)-1: # es ist ein zusätzlicher whitespace
                new_string += '[WHITESPACE] '  # sonst wird der doppelte Whitespace durch split() ignoriert
            else:
                new_string += b
        else: # b nicht ' ' und kein Punkt
            if i>0 and string[i-1] in substrings: 
                new_string += ' [ANTISPACE] '
                new_string += b
            else:
                new_string += b
    return new_string
    

def de_punctuation(string):
    """
    Macht punctuation komplett wieder rückgängig.
    """
    string = string.replace(' [ANTISPACE] ','')
    string = string.replace('[WHITESPACE]','')  # [WHITESPACE] liefert den whitespace wie jedes Token mit
    
    string = string.replace('∆','\r\n\r\n') # multi bytes ersetzen
    string = string.replace('Ω','\r\n') # multi bytes ersetzen
    string = string.replace('ª','\n') # multi bytes ersetzen
    string = string.replace('µ','\r') # multi bytes ersetzen
    
    return string


def split(string):
    return string.split()

def de_split(liste):
    string = ' '.join(liste)
    if string.endswith('[WHITESPACE]'):
        string += ' '
    return string

def capitalize(liste, most_common_capitalization):
    new_liste = []
    for word in liste:
        upper = word[0].upper() + word[1:]
        lower = word[0].lower() + word[1:]
        
        if upper in most_common_capitalization:
            if word == upper:
                new_liste.append(upper)
            elif word == lower:
                new_liste.append('[LOWER]')
                new_liste.append(upper)
        else: # lower ist die Normalform
            if word == upper:
                new_liste.append('[UPPER]')
                new_liste.append(lower)
            elif word == lower:
                new_liste.append(lower)
    return new_liste

def de_capitalize(liste):
    new_liste = []
    up  = False  # vorheriges wort war lower token
    low = False  # vorheriges word war upper token
    
    for word in liste:
        if word == '[LOWER]':
            low = True
            up  = False
        elif word == '[UPPER]':
            up  = True
            low = False
        
        else:
            if up == True:
                new_liste.append(word[0].upper()+word[1:])
            elif low == True:
                new_liste.append(word[0].lower()+word[1:])
            else:
                new_liste.append(word)
            low = False
            up  = False
    return new_liste
                
def stem(liste, stemming_dict):
    new_liste = []
    suffixe = ['en','st','es','em','er','e','t']
    for word in liste:
        no_suffix = True
        for suffix in suffixe:
            if word.endswith(suffix) and word[:-1*len(suffix)] in stemming_dict:
                new_liste.append(word[:-1*len(suffix)])
                new_liste.append('§$'+suffix)
                no_suffix = False
                break
        if no_suffix:
            new_liste.append(word)
    return new_liste

def de_stem(liste):
    new_liste = []
    for token in liste:
        if len(token)>1 and token[:2]=='§$' and new_liste:
            new_liste[-1] += token[2:]
        else:
            new_liste.append(token)
    return new_liste

def numberize(liste, token_to_number):
    return [token_to_number[token] if token in token_to_number else token_to_number['[UKN]'] for token in liste]

def de_numberize(liste, number_to_token):
    return [number_to_token[number] for number in liste]

In [13]:
class Tokenizer:
    """
        Turns recipes into sequences of numbers
    """
    
    def __init__(self, df, token_no=10000, validation_size=1000):
        
        # Rezepte erstellen:
        self.recipes       = []
        self.extract_recipes(df)
        random.Random(0).shuffle(self.recipes)
        
        self.validation_size = validation_size
        
        self.token_number      = token_no
        
        self.suffixe       = ['en','st','es','em','er','e','t']
        self.suffix_count  = {'en':0, 'st':0, 'es':0, 'em':0, 'er':0, 'e':0, 't':0}
        self.symbols       = ['.', '!', '?', '∆',':','Ω','ª','µ'] 
        
        
        self.word_count = {}
        self.stemmed_count    = {}
        self.counting()
        
        self.most_common_capitalization = {}
        self.most_common_case()
        
        self.stemming_dict = {}
        self.token_to_number = {}
        self.number_to_token = []
        self.compute_stems()
        self.compute_tokens()
        
        # saving precomputed data for tokenization: 
        self.precomputed_dicts = (self.stemming_dict,
                                  self.most_common_capitalization,
                                  self.token_to_number,
                                  self.number_to_token)
        json_string = json.dumps(self.precomputed_dicts)
        with open('precomputed_dicts.json','w') as f:
            f.write(json_string)
        
        
    def extract_recipes(self, df):
        """
            Extracts recipes from the dataframe 
        """
        for r in range(len(df)):
            try: 
                ingredients  = df.recipeIngredient.iloc[r][2:-2].split("', '")
                instructions = df.recipeInstructions.iloc[r]
                recipe = '\r\n\r\n'.join(ingredients) + '\r\n\r\n' + ' [INSTRUCTIONS] ' + instructions
                
                recipe = recipe.replace('\u2028',' ') # Traurige Notwendigkeit
                recipe = recipe.replace('\u2009',' ') # Traurige Notwendigkeit
                recipe = recipe.replace('\u2003',' ') # Traurige Notwendigkeit
                recipe = recipe.replace('\u2007',' ') # Traurige Notwendigkeit
                recipe = recipe.replace('\u2005',' ') # Traurige Notwendigkeit
                recipe = recipe.replace('\u202f',' ') # Traurige Notwendigkeit
                recipe = recipe.replace('\u2029',' ') # Traurige Notwendigkeit
                recipe = recipe.replace('\t',' ')     # Traurige Notwendigkeit
                recipe = recipe.replace('\xa0',' ')   # Traurige Notwendigkeit
                recipe = recipe.replace('\x1e',' ')   # Traurige Notwendigkeit
                recipe = recipe.replace('\x0b',' ')   # Traurige Notwendigkeit
                
                self.recipes.append(recipe)
            except:
                print("Skipping recipe",r)
                print(df.loc[r].recipeIngredient)
                print(df.loc[r].recipeInstructions)
                print()
        print("{} recipes extracted.".format(len(self.recipes)))
        
        
    def counting(self):
        """
        Hier werden direkt naive_words, stemmed_parts, capitalization gezählt
        Und dann das stemming_dict + token_dict etc berechnet.
        """

        start_time = time.time()

        # wörter und stemmed wörter werden gezählt
        for recipe in tqdm(self.recipes):
            punc = punctuation(recipe)
            naive_words = punc.split()
            for i,word in enumerate(naive_words):
                if word in self.word_count:
                    self.word_count[word] += 1
                else:
                    self.word_count[word]  = 1

                for suffix in self.suffixe:
                    if len(word)-len(suffix)>2:
                        if word.endswith(suffix):
                            self.suffix_count[suffix] += 1
                            if word[:-1*len(suffix)] in self.stemmed_count:
                                self.stemmed_count[word[:-1*len(suffix)]] += 1
                            else:
                                self.stemmed_count[word[:-1*len(suffix)]]  = 1
        print("{} minutes for counting words.".format((time.time()-start_time)/60))   
        
    def most_common_case(self):
        # Ist Groß- oder Kleinschreibung der Normalfall für ein Wort?
        for word in self.word_count:
            upper = word[0].upper() + word[1:]
            lower = word[0].lower() + word[1:]
            upcount  = 0
            lowcount = 0
            if upper in self.word_count:
                upcount = self.word_count[upper]
            if lower in self.word_count:
                lowcount = self.word_count[lower]
            if lower == upper: 
                lowcount = 0 # sonst wird doppelt gezählt

            if upcount > lowcount:
                self.most_common_capitalization[upper] = upcount + lowcount
            else:
                self.most_common_capitalization[lower] = upcount + lowcount
        
        print('Most common capitalization computed.')
                
                
    def compute_stems(self):
        """
        Wir vergleichen den count von stems mit dem count des ganzen Wortes
        Falls der stem doppelt so häufig ist, sagen wir das stemming eine gute Idee ist.
        D.h. das der stem in das stemming_dict kommt.
        """
        count = 0
        for word in self.most_common_capitalization:  # wir brauchen nur eine schreibweise
            for suffix in self.suffixe:
                if word.endswith(suffix) and len(word)-len(suffix)>2:
                    if self.stemmed_count[word[:-1*len(suffix)]] > 2*self.most_common_capitalization[word]:
                        self.stemming_dict[word[:-1*len(suffix)]] = True
                        break  # Nur eine Form des Stemming 'st' statt 't' z.B.
        # Jetzt kennen wir das stemming und zählen wörter neu:
        self.word_count = {}
        for word in self.most_common_capitalization:  # wir brauchen nur eine schreibweise
            potentially_stemmed_word = word
            for suffix in self.suffixe:
                if word.endswith(suffix) and word[:-1*len(suffix)] in self.stemming_dict:
                    potentially_stemmed_word = word[:-1*len(suffix)]
                    break  # Nur eine Form des Stemming 'st' statt 't' z.B.
            if potentially_stemmed_word in self.word_count: # stems kommen öfter vor
                self.word_count[potentially_stemmed_word] += self.most_common_capitalization[word]
            else:
                self.word_count[potentially_stemmed_word]  = self.most_common_capitalization[word]
        # Dieses word_count enthält nun die tokens nach capitalization und stemming
        print(len(self.stemmed_count),'stems chosen.')
        
    def compute_tokens(self):
        """
        Hier zerlegen wir die gezählten Wörter mit stemming und in den most_common_capitalization case
        Dann sortieren wir nach count und nehmen die häufigsten dieser Teile ins token_dict auf.
        """

        self.word_count = dict(sorted(self.word_count.items(), key=lambda item: item[1])[::-1][:self.token_number-12])
        self.token_to_number = {'[UKN]':0,'[STOP]':1,'[UPPER]':2,'[LOWER]':3,'[INSTRUCTIONS]':4}
        count = 5

        self.number_to_token = ['[UKN]','[STOP]','[UPPER]','[LOWER]','[INSTRUCTIONS]']

        # suffixe für das stemming zuerst.
        for token in ['§$en','§$st','§$es','§$em','§$er','§$e','§$t']: 
            self.token_to_number[token] = count
            self.number_to_token.append(token)
            count += 1

        for token in self.word_count:
            if token not in self.token_to_number:
                self.token_to_number[token] = count
                self.number_to_token.append(token)
                count += 1

        print(len(self.token_to_number),'tokens chosen.')
        
        
    def tokenize(self,string):
        punc = punctuation(string)
        liste = split(punc)
        capped_liste = capitalize(liste,self.most_common_capitalization)
        stemmed = stem(capped_liste,self.stemming_dict)
        tokens = numberize(stemmed,self.token_to_number)
        return tokens

    def de_tokenize(self,tokens):
        liste  = de_numberize(tokens,self.number_to_token)
        liste  = de_stem(liste)
        liste  = de_capitalize(liste)
        string = de_split(liste)
        string = de_punctuation(string)
        return string


    def generator(self, batchsize, seqlen):
        batch_list = []
        while True:
            for recipe in self.recipes[self.validation_size:]:
                tokens = self.tokenize(recipe)
                tokens = tokens + [self.token_to_number['[STOP]']] * seqlen
                tokens = np.array(tokens)
                tokens = tokens.reshape(-1,len(tokens))
                x = tokens[:,:seqlen]
                y = tokens[:,1:seqlen+1]
                batch_list.append((x,y))
                if len(batch_list) == batchsize:
                    xx = np.concatenate([x for (x,y) in batch_list])
                    yy = np.concatenate([y for (x,y) in batch_list])
                    batch_list = []
                    yield (xx,yy)

    def validation_generator(self, batchsize, seqlen):
        batch_list = []
        for recipe in self.recipes[:self.validation_size]:
            tokens = self.tokenize(recipe)
            tokens = tokens + [self.token_to_number['[STOP]']] * seqlen
            tokens = np.array(tokens)
            tokens = tokens.reshape(-1,len(tokens))
            x = tokens[:,:seqlen]
            y = tokens[:,1:seqlen+1]
            batch_list.append((x,y))
            if len(batch_list) == batchsize:
                xx = np.concatenate([x for (x,y) in batch_list])
                yy = np.concatenate([y for (x,y) in batch_list])
                batch_list = []
                yield (xx,yy)
                

In [14]:
df = pd.read_csv('10000_Chefkoch_Rezepte.csv') 
tokey = Tokenizer(df)
df = '' # Memory freigeben

10000 recipes extracted.


100%|████████████████████████████████████| 10000/10000 [00:11<00:00, 846.57it/s]


0.19690123399098713 minutes for counting words.
Most common capitalization computed.
15581 stems chosen.
9999 tokens chosen.


In [15]:
for batch in tokey.generator(32,512):
    print(batch[0].shape,batch[1].shape)
    break

(32, 512) (32, 512)


In [16]:
count = 0
for key in tokey.word_count:
    print(key,tokey.word_count[key])
    count += 1
    if count == 100:
        break
    

[ANTISPACE] 604578
. 131552
∆ 116151
, 97698
und 80510
die 55876
mit 35504
ª 35384
in 34738
( 34594
) 34588
1 31239
den 27962
g 27877
ein 27514
n 23172
2 17852
[WHITESPACE] 17697
der 17046
- 16248
EL 15345
das 14534
Salz 12829
auf 11985
lass 11539
dem 11077
[INSTRUCTIONS] 10000
Pfeffer 9984
oder 9430
geb 9246
schneid 9038
Minut 8675
ca 8576
etwas 8537
3 7774
Zuck 7520
TL 7124
ml 6989
klein 6643
/ 6454
nach 6284
4 6256
Wasser 6026
dann 6015
im 5985
Butt 5872
für 5821
200 5478
bei 5412
Ω 5377
Öl 4960
zu 4915
groß 4874
bis 4797
Scheib 4738
zum 4671
100 4667
Zwiebel 4622
Mehl 4551
fein 4511
aus 4500
Sahne 4463
gut 4192
5 4157
Pfann 4068
ist 4011
° 3955
½ 3911
Olivenöl 3826
Tomat 3775
schäl 3710
B 3656
Teig 3623
Knoblauch 3621
C 3576
frisch 3536
Min 3509
all 3431
wasch 3389
Ei 3369
250 3272
verteil 3239
back 3187
koch 3124
Milch 3112
noch 3094
abschmeck 3091
er 3080
von 3045
Stück 3037
servier 3036
Backofen 3004
unt 2998
10 2933
150 2882
dazugeb 2844
rühr 2829
50 2815
auch 2748
nicht 2744


In [17]:
# test was in word_count drin ist: [INSTRUCTIONS] etc
tokey.word_count['Reis']


1254

In [18]:
for recipe in tokey.recipes:
    tokens = tokey.tokenize(recipe)
    decode = tokey.de_tokenize(tokens)
    print(recipe)
    print(decode)
    break

120 g Kartoffel(n)

 Salz

 Bohnenkraut

65 g Brechbohnen , TK

1 kleine Birne(n)

15 g Katenschinken , gewürfelt

30 g Schmand

15 g Milch

1  Ei(er)

 Pfeffer , schwarz, aus der Mühle

20 g Emmentaler , grob gerieben

1 TL Rapsöl zum Einfetten der Form

 [INSTRUCTIONS] Die Kartoffeln schälen und klein würfeln. Die Würfel in einem Topf mit Salzwasser und etwas Bohnenkraut einmal aufkochen, dann die Bohnen dazugeben und alles abgedeckt 10 Minuten bei kleinerer Hitze köcheln lassen. Nach dieser Zeit das Gemüse über einem Sieb abgießen, mit kaltem Wasser abschrecken und zum Abtropfen über eine Schüssel hängen.

Den Ofen auf 180 Grad vorheizen.

In einer Schüssel Schmand, Milch und das Ei verrühren, mit Salz und Pfeffer abschmecken und etwas Bohnenkraut untermischen. Eine kleine Auflaufform einfetten.

Kartoffeln und Bohnen unter die Schmandmischung heben und den Katenschinken untermischen. Die Birne schälen, entkernen, klein würfeln und untermengen. Die Masse in die Auflaufform umfüllen 

Das counting ist kompliziert, weil es sich nach jeden zusätzlich berechneten preprocessing schritt verändert. D.h. nach dem Stemming sind die Stems neu und die ganzen Wörter obsolet. Nach dem most_common_capitalization Schritt gibt es nur noch eine Schreibweise, aber der count der anderen Schreibweise ist noch relevant. 

Wie muss man das aufziehen? 
* capitalize kommt vor dem stemming d.h. stemming muss auf der einen Schreibweise basieren. 
* D.h. die naive_word_count wird einfach word_count und hier umgeformt zu einer Schreibweise mit Kombicount.
* Dann wird stemming_dict berechnet.
* stems müssen zum word_count hinzugefügt werden, gestemmte Wörter entfernt.
* das sollte durch Umformen der dicts passieren, nicht durch neu-tokenisieren der Rezepte.

In [19]:
### Transformer Code

In [20]:
def causal_attention_mask(batch_size, n_dest, n_src, dtype):
    """
    Mask the upper half of the dot product matrix in self attention.
    This prevents flow of information from future tokens to current token.
    1's in the lower triangle, counting from the lower right corner.
    """
    i = tf.range(n_dest)[:, None]
    j = tf.range(n_src)
    m = i >= j - n_src + n_dest
    mask = tf.cast(m, dtype)
    mask = tf.reshape(mask, [1, n_dest, n_src])
    mult = tf.concat(
        [tf.expand_dims(batch_size, -1), tf.constant([1, 1], dtype=tf.int32)], 0
    )
    return tf.tile(mask, mult)


class TransformerBlock(layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super(TransformerBlock, self).__init__()
        self.att = layers.MultiHeadAttention(num_heads, embed_dim)
        self.ffn = keras.Sequential(
            [layers.Dense(ff_dim, activation="relu"), layers.Dense(embed_dim),]
        )
        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = layers.Dropout(rate)
        self.dropout2 = layers.Dropout(rate)
        
        # rezero addition
        self.alpha = tf.Variable( initial_value=1e-4, dtype="float32",  trainable=True)

    def call(self, inputs):
        input_shape = tf.shape(inputs)
        batch_size = input_shape[0]
        seq_len = input_shape[1]
        causal_mask = causal_attention_mask(batch_size, seq_len, seq_len, tf.bool)
        attention_output = self.att(inputs, inputs, attention_mask=causal_mask)
        attention_output = self.dropout1(attention_output)
        out1 = self.layernorm1(inputs + self.alpha * attention_output) # rezero addition
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output)
        return self.layernorm2(out1 + self.alpha * ffn_output) # rezero addition
    
class TokenAndPositionEmbedding(layers.Layer):
    def __init__(self, maxlen, vocab_size, embed_dim):
        super(TokenAndPositionEmbedding, self).__init__()
        self.token_emb = layers.Embedding(input_dim=vocab_size, output_dim=embed_dim)
        self.pos_emb = layers.Embedding(input_dim=maxlen, output_dim=embed_dim)

    def call(self, x):
        maxlen = tf.shape(x)[-1]
        positions = tf.range(start=0, limit=maxlen, delta=1)
        positions = self.pos_emb(positions)
        x = self.token_emb(x)
        return x + positions

In [21]:
vocab_size = 10000  # Only consider the top 10k words
maxlen = 512  # Max sequence size
embed_dim = 256  # Embedding size for each token
num_heads = 2  # Number of attention heads
feed_forward_dim = 256  # Hidden layer size in feed forward network inside transformer


def create_model(no_of_layers):
    inputs = layers.Input(shape=(maxlen,), dtype=tf.int32)
    embedding_layer = TokenAndPositionEmbedding(maxlen, vocab_size, embed_dim)
    x = embedding_layer(inputs)
    
    for t in range(no_of_layers):
        transformer_block = TransformerBlock(embed_dim, num_heads, feed_forward_dim)
        x = transformer_block(x)
    
    outputs = layers.Dense(vocab_size)(x)
    model = keras.Model(inputs=inputs, outputs=[outputs, x])
    loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
    model.compile(
        "adam", loss=[loss_fn, None],
    )  # No loss and optimization based on word embeddings from transformer block
    return model


In [22]:
class TextGenerator(keras.callbacks.Callback):
    """A callback to generate text from a trained model.
    1. Feed some starting prompt to the model
    2. Predict probabilities for the next token
    3. Sample the next token and add it to the next input

    Arguments:
        max_tokens: Integer, the number of tokens to be generated after prompt.
        start_tokens: List of integers, the token indices for the starting prompt.
        index_to_word: List of strings, obtained from the TextVectorization layer.
        top_k: Integer, sample from the `top_k` token predictions.
        print_every: Integer, print after this many epochs.
    """

    def __init__(
        self, max_tokens, start_tokens, index_to_word, top_k=10, print_every=1
    ):
        self.max_tokens = max_tokens
        self.start_tokens = start_tokens
        self.index_to_word = index_to_word
        self.print_every = print_every
        self.k = top_k

    def sample_from(self, logits):
        logits, indices = tf.math.top_k(logits, k=self.k, sorted=True)
        indices = np.asarray(indices).astype("int32")
        preds = keras.activations.softmax(tf.expand_dims(logits, 0))[0]
        preds = np.asarray(preds).astype("float32")
        return np.random.choice(indices, p=preds)

    def detokenize(self, number):
        return self.index_to_word[number]
    
    def de_tokenize(self,tokens):
        liste  = de_numberize(tokens,self.index_to_word)
        liste  = de_stem(liste)
        liste  = de_capitalize(liste)
        string = de_split(liste)
        string = de_punctuation(string)
        return string

    def on_epoch_end(self, epoch, logs=None):
        start_tokens = [_ for _ in self.start_tokens]
        if (epoch + 1) % self.print_every != 0:
            return
        num_tokens_generated = 0
        tokens_generated = []
        while num_tokens_generated <= self.max_tokens:
            pad_len = maxlen - len(start_tokens)
            sample_index = len(start_tokens) - 1
            if pad_len < 0:
                x = start_tokens[:maxlen]
                sample_index = maxlen - 1
            elif pad_len > 0:
                x = start_tokens + [0] * pad_len
            else:
                x = start_tokens
            x = np.array([x])
            y, _ = self.model.predict(x)
            sample_token = self.sample_from(y[0][sample_index])
            tokens_generated.append(sample_token)
            start_tokens.append(sample_token)
            num_tokens_generated = len(tokens_generated)
            
        txt = " ".join(
            [self.detokenize(_) for _ in self.start_tokens + tokens_generated]
        )
        txt = txt.split('[STOP]')[0]
        line_liste = txt.split('\r\n')
        print(f"generated text:\n")
        for line in line_liste:
            print(line)
        print()
        
        print("The other detokenizer ...")
        txt = self.de_tokenize(self.start_tokens + tokens_generated)
        txt = txt.split('[STOP]')[0]
        line_liste = txt.split('\r\n')
        for line in line_liste:
            print(line)
        print()
        
# Tokenize starting prompt

start_prompt = "250 g Reis"
start_tokens = [tokey.token_to_number.get(_, 1) for _ in start_prompt.split()]
num_tokens_generated = 250
text_gen_callback = TextGenerator(num_tokens_generated, start_tokens, tokey.number_to_token)



In [25]:
model = create_model(4)
model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 512)]             0         
                                                                 
 token_and_position_embeddin  (None, 512, 256)         2691072   
 g_1 (TokenAndPositionEmbedd                                     
 ing)                                                            
                                                                 
 transformer_block_16 (Trans  (None, 512, 256)         658689    
 formerBlock)                                                    
                                                                 
 transformer_block_17 (Trans  (None, 512, 256)         658689    
 formerBlock)                                                    
                                                                 
 transformer_block_18 (Trans  (None, 512, 256)         6586

In [ ]:
history = model.fit(
    tokey.generator(32,512), 
    verbose=1, 
    steps_per_epoch = 100,
    callbacks = [text_gen_callback],
    epochs=100)

Epoch 1/100
100/100 [==============================] - ETA: 0s - loss: 2.9621 - dense_41_loss: 2.9621generated text:

250 g Reis , ( [ANTISPACE] s [ANTISPACE] ) [ANTISPACE] ∆ [ANTISPACE] ∆ [ANTISPACE] ∆ [ANTISPACE] ) [ANTISPACE] 1 Dos §$e [ANTISPACE] n [ANTISPACE] ∆ [ANTISPACE] ) [ANTISPACE] ∆ [ANTISPACE] , ( [ANTISPACE] n [ANTISPACE] n [ANTISPACE] ∆ [ANTISPACE] ) [ANTISPACE] ∆ [ANTISPACE] / [ANTISPACE] ∆ [ANTISPACE] , gehack §$t [ANTISPACE] ∆ [ANTISPACE] 1 EL Sojasauc §$e [ANTISPACE] ∆ [ANTISPACE] ) , fein gehack §$t [ANTISPACE] ( [ANTISPACE] . [ANTISPACE] 2 cl [ANTISPACE] ∆ [ANTISPACE] ( [ANTISPACE] n [ANTISPACE] n [ANTISPACE] , rot [ANTISPACE] ∆ [ANTISPACE] n [ANTISPACE] ∆ Salz und mit dem Öl [ANTISPACE] . [ANTISPACE] ) [ANTISPACE] 1 [WHITESPACE] Zitron §$e in die Sauc §$e , den Ofen nehm §$en [ANTISPACE] - [ANTISPACE] . B [ANTISPACE] . [UPPER] die Banan §$en und die Zwiebeln [ANTISPACE] . [UPPER] dann die Banan §$en lass §$en und all §$es [ANTISPACE] ª [ANTISPACE] ∆ [ANTISPACE] , b

Epoch 5/100
100/100 [==============================] - ETA: 0s - loss: 1.5799 - dense_41_loss: 1.5799generated text:

250 g Reis [ANTISPACE] / [ANTISPACE] s [ANTISPACE] ∆ [ANTISPACE] ∆ [ANTISPACE] ∆ [ANTISPACE] 2 Pck [ANTISPACE] 250 g Butt §$er , weiß [ANTISPACE] . Sahnesteif [ANTISPACE] ∆ [ANTISPACE] ∆ [ANTISPACE] ∆ [ANTISPACE] 4 [WHITESPACE] Zitron §$e [ANTISPACE] ( [ANTISPACE] ) [ANTISPACE] 1 Pck [ANTISPACE] 1 Pris §$e [ANTISPACE] ∆ [ANTISPACE] , gehack §$t [ANTISPACE] ∆ [ANTISPACE] 2 [WHITESPACE] Knoblauchzeh §$e , ( [ANTISPACE] ) [ANTISPACE] ∆ [ANTISPACE] . B [ANTISPACE] ∆ [ANTISPACE] 150 g Butt §$er , gehäuft §$e [ANTISPACE] n [ANTISPACE] ∆ [ANTISPACE] etwas Öl ( [ANTISPACE] ) , ( [ANTISPACE] ∆ [INSTRUCTIONS] [UPPER] die Zwiebel schäl §$en [ANTISPACE] , in Salzwasser koch §$en und in ein §$em Sieb abtropf §$en [ANTISPACE] , Knoblauch und mit Zuck §$er Schüssel geb §$en und Pfeffer würz §$en [ANTISPACE] . [UPPER] mit etwas Öl anbraten [ANTISPACE] ª [ANTISPACE] . ª [ANTISPACE] . [U